In [98]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
import numpy as np

## Counties and county equivalents

In [99]:
df_counties = pd.read_csv(
    "../data/raw/fips_lookup.csv",
    dtype={"state_fips": str, "county_fips": str}
).rename(columns={"county_name": "area"})

df_counties["fips"] = df_counties.state_fips + df_counties.county_fips
df_counties = df_counties.loc[:, ["fips", "area"]]
df_counties.sort_values("fips", axis=0, inplace=True)

_, counts = np.unique(df_counties.fips.values, return_counts=True)
print(f"All FIPS unique: {np.all(counts == 1)}")

df_counties.head()

,fips,area
29,01001,Autauga County
30,01003,Baldwin County
31,01005,Barbour County
32,01007,Bibb County
33,01009,Blount County


## Unemployment

In [117]:
df_unemp = pd.read_excel(
    "../data/raw/Unemployment.xlsx", 
    sheet_name="Unemployment Med HH Income", 
    skiprows=4, 
    usecols=["FIPS_Code", "Area_name", "Unemployment_rate_2020", "Median_Household_Income_2019"],
    dtype={"FIPS_Code": str}
).rename(columns={"FIPS_Code": "fips", "Area_name": "area"})

df_unemp.columns = df_unemp.columns.str.lower()
# df_unemp.insert(1, "county", df_unemp.area_name.str.extract(r"(.*(?=County,\s[A-Z]{2}$))")[0])
# df_unemp.drop(columns=["area_name"], inplace=True)
# df_unemp.dropna(subset=["county"], inplace=True)

print(df_unemp.shape)

_, counts = np.unique(df_unemp.fips.values, return_counts=True)
print(f"All FIPS unique: {np.all(counts == 1)}")

df_unemp.head()

(3275, 4)
All FIPS unique: True


,fips,area,unemployment_rate_2020,median_household_income_2019
0,00000,United States,8.05281,65712.0
1,01000,Alabama,5.90000,51771.0
2,01001,"Autauga County, AL",4.90000,58233.0
3,01003,"Baldwin County, AL",5.60000,59871.0
4,01005,"Barbour County, AL",7.00000,35972.0


## Population

In [101]:
import chardet

with open("../data/raw/co-est2019-alldata.csv", "rb") as file:
    best_encoding = chardet.detect(file.read())

print(best_encoding)


{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


In [118]:
df_pop = pd.read_csv(
    "../data/raw/co-est2019-alldata.csv",
    usecols=["STATE", "COUNTY", "CTYNAME", "POPESTIMATE2019"],
    dtype={"STATE": str, "COUNTY": str},
    encoding=best_encoding["encoding"]
).rename(columns={"POPESTIMATE2019": "pop_estimate_2019", "CTYNAME": "area"})

# df_pop.rename(columns={"POPESTIMATE2019": "pop_estimate_2019"}, inplace=True)
# df_pop["county"] = df_pop.CTYNAME.str.extract(r"(.*(?=\sCounty))")[0]
df_pop["fips"] = df_pop.STATE.values + df_pop.COUNTY.values
df_pop = df_pop.loc[:, ["fips", "area", "pop_estimate_2019"]]
# df_pop.dropna(subset=["county"], inplace=True)

print(df_pop.shape)

_, counts = np.unique(df_pop.fips.values, return_counts=True)
print(f"All FIPS unique: {np.all(counts == 1)}")

df_pop.head()

(3193, 3)
All FIPS unique: True


,fips,area,pop_estimate_2019
0,01000,Alabama,4903185
1,01001,Autauga County,55869
2,01003,Baldwin County,223234
3,01005,Barbour County,24686
4,01007,Bibb County,22394


## GDP per capita

We don't need it as we already have median household income + this is the only one dataset without fips code available.

In [103]:
# df_gdp = pd.read_excel(
#     "../data/raw/lagdp1221.xlsx",
#     sheet_name="Table 1",
#     skiprows=3,
#     usecols=[0, 4, 5],
#     na_values="--"
# )
# df_gdp.columns = ["area", "gdp", "rank_in_state"]
# df_gdp.dropna(inplace=True)
# df_gdp.drop(columns=["rank_in_state"], inplace=True)

# print(df_gdp.shape)
# df_gdp.head()

## Election

In [139]:
df_election = pd.read_csv(
    "../data/raw/countypres_2000-2020.csv",
    usecols=["year", "county_name", "county_fips", "candidate", "party", "candidatevotes", "totalvotes"],
    dtype={"county_fips": str}
).rename(columns={"county_name": "area", "county_fips": "fips"})

df_election = df_election.loc[(df_election.year == 2020) & (df_election.candidate != "OTHER"), :]
df_election["votes"] = df_election.candidatevotes / df_election.totalvotes
df_election["area"] = df_election.area.str.title()
df_election.drop(columns=["year", "candidatevotes", "totalvotes"], inplace=True)

df_election.head()

,area,fips,candidate,party,votes
50524,Autauga,01001,JOSEPH R BIDEN JR,DEMOCRAT,0.270184
50526,Autauga,01001,DONALD J TRUMP,REPUBLICAN,0.714368
50527,Baldwin,01003,JOSEPH R BIDEN JR,DEMOCRAT,0.224090
50529,Baldwin,01003,DONALD J TRUMP,REPUBLICAN,0.761714
50530,Barbour,01005,JOSEPH R BIDEN JR,DEMOCRAT,0.457882


In [143]:
df_votes_democrats = df_election.loc[df_election.party == "DEMOCRAT", ["fips", "area", "votes"]]\
    .rename(columns={"votes": "voted_biden"}) \
    .groupby(["fips", "area"], as_index=False) \
    .sum() \

fipses, counts = np.unique(df_votes_democrats.fips.values.astype(str), return_counts=True)
print(f"All FIPS unique: {np.all(counts == 1)}")

df_votes_democrats.head()

All FIPS unique: True


,fips,area,voted_biden
0,01001,Autauga,0.270184
1,01003,Baldwin,0.224090
2,01005,Barbour,0.457882
3,01007,Bibb,0.206983
4,01009,Blount,0.095694


## Poverty

In [144]:
df_poverty = pd.read_excel(
    "../data/raw/PovertyEstimates.xls",
    usecols=["FIPStxt", "Area_name", "PCTPOVALL_2019"],
    dtype={"FIPStxt": str},
    skiprows=4
).rename(columns={
    "FIPStxt": "fips",
    "Area_name": "area",
    "PCTPOVALL_2019": "poverty_frac_2019"
})

df_poverty["poverty_frac_2019"] = df_poverty.poverty_frac_2019 / 100

print(df_poverty.shape)

fipses, counts = np.unique(df_poverty.fips.values.astype(str), return_counts=True)
print(f"All FIPS unique: {np.all(counts == 1)}")

df_poverty.head()

(3193, 3)
All FIPS unique: True


,fips,area,poverty_frac_2019
0,00000,United States,0.123
1,01000,Alabama,0.156
2,01001,Autauga County,0.121
3,01003,Baldwin County,0.101
4,01005,Barbour County,0.271


## Education

In [145]:
df_education = pd.read_excel(
    "../data/raw/Education.xls",
    usecols=[
        "FIPS Code", 
        "Area name", 
        "Percent of adults with less than a high school diploma, 2015-19", 
        "Percent of adults with a high school diploma only, 2015-19",
        "Percent of adults completing some college or associate's degree, 2015-19",
        "Percent of adults with a bachelor's degree or higher, 2015-19"
    ],
    dtype={"FIPS Code": str},
    skiprows=4
)

for c in df_education.columns:
    if is_numeric_dtype(df_education[c]):
        df_education[c] = df_education[c] / 100

df_education.columns = ["fips", "area", "no_high_school", "high_school_only", "college_only", "bachelor_or_higher"]

print(df_education.shape)

fipses, counts = np.unique(df_education.fips.values.astype(str), return_counts=True)
print(f"All FIPS unique: {np.all(counts == 1)}")

df_education.head()


(3283, 6)
All FIPS unique: True


,fips,area,no_high_school,high_school_only,college_only,bachelor_or_higher
0,00000,United States,0.119989,0.269568,0.288987,0.321455
1,01000,Alabama,0.138193,0.308003,0.299121,0.254683
2,01001,Autauga County,0.114834,0.335885,0.283566,0.265716
3,01003,Baldwin County,0.091938,0.276596,0.312841,0.318625
4,01005,Barbour County,0.267869,0.356045,0.260298,0.115787


## Combining

In [149]:
df_full = df_counties.copy()

for df in [df_pop, df_unemp, df_poverty, df_education, df_votes_democrats]:
    df_full = pd.merge(
         df_full,
         df.drop(columns=["area"]),
         on="fips",
         how="left"
     )

print(df_full.shape)

df_full.head()

(3233, 11)


,fips,area,pop_estimate_2019,unemployment_rate_2020,median_household_income_2019,poverty_frac_2019,no_high_school,high_school_only,college_only,bachelor_or_higher,voted_biden
0,01001,Autauga County,55869.0,4.9,58233.0,0.121,0.114834,0.335885,0.283566,0.265716,0.270184
1,01003,Baldwin County,223234.0,5.6,59871.0,0.101,0.091938,0.276596,0.312841,0.318625,0.224090
2,01005,Barbour County,24686.0,7.0,35972.0,0.271,0.267869,0.356045,0.260298,0.115787,0.457882
3,01007,Bibb County,22394.0,6.6,47918.0,0.203,0.209426,0.448788,0.238001,0.103785,0.206983
4,01009,Blount County,57826.0,4.1,52902.0,0.163,0.195094,0.334221,0.339750,0.130934,0.095694


In [148]:
df_full.dropna().shape[0]

3113

In [147]:
df_full.to_csv("../data/non_epidemic_data.csv", index=False)